In [1]:
import pandas as pd
import json
import bs4
import re
from tqdm.notebook import tqdm

In [2]:
# import os

# datafiles = []
# for root, dirs, files in os.walk('../data_scraping/'):
#     for file in files:
#         if file.endswith('.pkl'):
#             datafiles.append(root + file)
# print(datafiles)

# pd.read_pickle(datafiles[0], compression=None)

In [3]:
# import zipfile  

# with zipfile.ZipFile('../data_scraping/bills.zip', 'r') as z:
#     for filename in z.namelist():   
#         with z.open(filename) as f:  
#             data = f.read()  
#             d = json.loads(data.decode("utf-8"))

In [4]:
import bz2

with bz2.open('../data_scraping/bills.json.bz2', 'r') as f:  
    data = f.read()  
    d = json.loads(data.decode("utf-8"))

In [5]:
# list(d.keys())[0]
# list(d.values())[0]
# d['http://www.senate.gov.ph/lis/bill_res.aspx?congress=13&q=SBN-1']
soup = bs4.BeautifulSoup(list(d.values())[15640], 'html.parser')
print(soup.prettify())

<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   18th Congress - Senate Bill No. 1736 - Senate of the Philippines
  </title>
  <script language="javascript" src="/scripts_web/div.js" type="text/javascript">
  </script>
  <link href="/scripts_web/print.css" media="print" rel="stylesheet" type="text/css"/>
  <link href="../scripts_web/lis.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="../scripts_web/xhtml.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="../scripts_web/xhtml_body.css" media="all" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <div class="xhtml_nav">
   <img alt="Senate of the Philippines banner" class="nav_logo" height="96" src="/images/website_banner18th.jpg" width="800"/>
   <div id="nav_top">
    <table>
     <tbody>
      <tr>
       <td>
        <a href="/">
         Home
        </a>
       </td>
       <td>
        |
       </td>
       

In [6]:
values = []

for i in tqdm(list(d.values())):
    dic = {}
    soup = bs4.BeautifulSoup(i, 'html.parser')
    content = soup.find('td', attrs = {'id':'content'}).text.strip()
    try:
        dic['congress'] = int(re.search('\d{2}(?=th Congress)',
                                        content).group(0))
        dic['type'] = re.search('(?<= Congress).+(?= Bill)', content).group(0)
        bill_num = int(re.search('(?<=Bill No. )\d{1,}', content).group(0))
        dic['bill_num'] = bill_num
        dic['title'] = re.search(f'(?<=Bill No. {bill_num}).+(?=Filed)',
                                 content).group(0)
        dic['filed_on'] = re.search('(?<=Filed on ).+(?= by)',
                                    content).group(0)
        dic['filed_by'] = re.search('(?<=by ).+', content).group(0)
        dic['introduced_by'] = re.search('(?<=Introduced by ).+',
                                         content).group(0)
        dic['long_title'] = re.search('(?<=Long title).+', content).group(0)
        dic['scope'] = re.search('(?<=Scope).+', content).group(0)
        dic['subjects'] = re.search('(?<=Subject\(s\)).+', content).group(0)
        dic['primary_committee'] = re.search('(?<=Primary committee).+',
                                             content).group(0)
        dic['secondary_committee'] = re.search('(?<=Secondary committee).+',
                                               content).group(0)
    except:
        pass
    values.append(dic)

In [7]:
df = pd.DataFrame(values)
df

,congress,type,bill_num,title,filed_on,filed_by,introduced_by,long_title,scope,subjects,primary_committee,secondary_committee
0,13,Senate,1,SPECIAL MEASURES TO LOWER THE COST OF MEDICINE,"June 30, 2004","Flavier, Juan M., Angara, Edgardo J.",Senator FLAVIER;,AN ACT PRESCRIBING SPECIAL MEASURES TO LOWER...,National,Medicines (Lowering the Cost) Drugs/...,Health and Demography,Trade and Commerce
1,13,Senate,2,THE CONSTITUTIONAL CONVENTION ACT,"June 30, 2004","Flavier, Juan M.",Senator FLAVIER;,AN ACT CALLING FOR A CONSTITUTIONAL CONVENTIO...,National,Constitutional Convention Charte...,"Constitutional Amendments, Revision of Codes a...",Finance
2,13,Senate,3,MAGNA CARTA OF PATIENT'S RIGHTS AND OBLIGATIONS,"June 30, 2004","Flavier, Juan M., Angara, Edgardo J.",Senator FLAVIER;,AN ACT DECLARING THE RIGHTS AND OBLIGATIONS O...,National,Patients Rights and Obligations Magna ...,Health and Demography,"Social Justice, Welfare and Rural Development"
3,13,Senate,4,HEALTH PROMOTION ACT OF 2004,"June 30, 2004","Flavier, Juan M., Angara, Edgardo J.",Senator FLAVIER;,AN ACT ESTABLISHING A NATIONAL HEALTH PROMOTI...,National,National Health Promotion Institute Philip...,Government Corporations and Public Enterprises,Health and Demography ...
4,13,Senate,5,INCREASING THE ALLOWABLE PERSONAL ADDITIONAL T...,"June 30, 2004","Flavier, Juan M.",Senator FLAVIER;,AN ACT INCREASING THE ALLOWABLE PERSONAL ADDIT...,National,Taxes (Tax Exemption-Individual TaxpayerNation...,Ways and Means,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15636,18,Senate,1732,CONVERTING THE ARAKAN-CALINAN MUNICIPAL ROAD I...,"July 23, 2020","Marcos, Imee R.",Senator IMEE R. MARCOS;,AN ACT CONVERTING THE ARAKAN-CALINAN MUNICIPAL...,Local,"Roads, Conversion",Rules,NaN
15637,18,Senate,1733,UNIVERSITY OF ABRA,"July 23, 2020","Marcos, Imee R.",Senator IMEE R. MARCOS;,AN ACT CONVERTING THE ABRA STATE INSTITUTE OF ...,Local,"Schools, Conversion","Higher, Technical and Vocational Education",Ways and MeansFinance
15638,18,Senate,1734,CAGAYAN VALLEY MEDICAL CENTER – ABULUG EXTENSI...,"July 23, 2020","Marcos, Imee R.",Senator IMEE R. MARCOS;,AN ACT ESTABLISHING A THIRTY (30) – BED DISTRI...,Local,"Hospitals, Establishment",Rules,NaN
15639,18,Senate,1735,BINENG NATIONAL HIGH SCHOOL,"July 23, 2020","Marcos, Imee R.",Senator IMEE R. MARCOS;,AN ACT CONVERTING THE BENGUET NATIONAL HIGH SC...,Local,"Schools, Conversion",Rules,NaN


In [8]:
df.isna().sum()

congress                  0
type                      0
bill_num                  0
title                     0
filed_on                  0
filed_by                  1
introduced_by           499
long_title              499
scope                   499
subjects                847
primary_committee       883
secondary_committee    5139
dtype: int64

In [9]:
df.dropna(subset=['subjects'], inplace=True)
df.to_csv('bills.csv')